In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
from analysis.lib.fitting import fit, common
from analysis.lib.pq import pq_tools;reload(pq_tools)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

thetas = ['pi/4','pi/5','pi/6','pi/8']


In [ ]:
### define parameters
start = 2755e3#2762.5e3
window_length = 125e3 # 50e3
hist_binsize = 100

In [ ]:
offsets_on = True

first_shifted = True
first_unshifted = True

for i,theta in enumerate(thetas):
    print 'Processing theta ', theta
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,shifted_days = [])
    print len(all_lt4)
    if np.size(all_lt4):
        if not(offsets_on):
            offsets = np.zeros(np.size(all_lt4))
            offsets_ch1 = np.zeros(np.size(all_lt4))
            
        (h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                              start = start,
                                                              length = window_length, 
                                                              hist_binsize = hist_binsize, 
                                                              offsets = offsets, 
                                                              offsets_ch1 = offsets_ch1)

        if first_shifted:
            ch0_hist_data_unshifted = h0
            ch1_hist_data_unshifted = h1
            first_shifted = False
        else:
            ch0_hist_data_unshifted += h0
            ch1_hist_data_unshifted += h1

            
    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,unshifted_days = [])
    print len(all_lt4)
    (h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                          start = start,
                                                          length = window_length, 
                                                          hist_binsize = hist_binsize)
    if np.size(all_lt4):
        if not(offsets_on):
            offsets = np.zeros(np.size(all_lt4))
            offsets_ch1 = np.zeros(np.size(all_lt4))
            
        (h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                              start = start,
                                                              length = window_length, 
                                                              hist_binsize = hist_binsize, 
                                                              offsets = offsets, 
                                                              offsets_ch1 = offsets_ch1)
        if first_unshifted:
            ch0_hist_data_shifted = h0
            ch1_hist_data_shifted = h1
            first_unshifted = False
        else:
            ch0_hist_data_shifted += h0
            ch1_hist_data_shifted += h1

In [ ]:
fig, (ax0) = plt.subplots(figsize=(12,4))
pa.plot_photon_hist(ax0, ch0_hist_data_unshifted, b0,label = 'ch0',log=True)
pa.plot_photon_hist(ax0, ch1_hist_data_unshifted, b1,label = 'ch1',log=True)
ax0.set_title('All data from unshifted days')
plt.show()
plt.close('all')

fig, (ax0) = plt.subplots(figsize=(12,4))
pa.plot_photon_hist(ax0, ch0_hist_data_shifted, b0,label = 'ch0',log=True)
pa.plot_photon_hist(ax0, ch1_hist_data_shifted, b1,label = 'ch1',log=True)
ax0.set_title('All data from shifted days')
plt.show()
plt.close('all')

In [ ]:
def xcorr(a,v):
            
    xaxis = np.arange(len(a)) - len(a)/2
    return xaxis,np.correlate(np.array(a)-np.mean(a),np.array(v)-np.mean(v),'same')


In [ ]:
xaxis,xcorr01_shifted=xcorr(ch0_hist_data_unshifted,ch1_hist_data_unshifted)
xaxis = (b0[1]-b0[0])*xaxis
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(xaxis,xcorr01_shifted)
plt.show()
plt.close('all')
i = np.argmax(xcorr01_shifted)
print 'Ch1 should be shifted by ', xaxis[i]

xaxis,xcorr01_shifted=xcorr(ch0_hist_data_shifted,ch1_hist_data_shifted)
xaxis = (b0[1]-b0[0])*xaxis
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(xaxis,xcorr01_shifted)
plt.show()
plt.close('all')
i = np.argmax(xcorr01_shifted)
print 'Ch1 should be shifted by ', xaxis[i]

total_unshifted_data = ch0_hist_data_unshifted + ch1_hist_data_unshifted
total_shifted_data = ch0_hist_data_shifted + ch1_hist_data_shifted

xaxis,xcorr01_shifted=xcorr(total_unshifted_data,total_shifted_data)
xaxis = (b0[1]-b0[0])*xaxis
fig, (ax0) = plt.subplots(figsize=(12,4))
plt.plot(xaxis,xcorr01_shifted)
plt.show()
plt.close('all')
i = np.argmax(xcorr01_shifted)
print 'Shifted days should be shifted by ', xaxis[i]


In [ ]:
hist_data = []

for theta in thetas:
    print 'Processing theta ', theta
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,unshifted_days = [])
    
    (h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4, offsets = offsets, offsets_ch1 = offsets_ch1,start = start,length = window_length, hist_binsize = hist_binsize)
    hist_data.append(h0 + h1)


In [ ]:
def fit_gaussian(ax,x,y,**kw):
    
     ### kw for fitting

    fit_offset = kw.pop('fit_offset',0)
    fit_amplitude = kw.pop('fit_amplitude', 2e4)
    fit_x0 = kw.pop('fit_x0', 2770)
    fit_sigma = kw.pop('fit_sigma', 5)
    fixed = kw.pop('fixed', [])
    show_guess = kw.pop('show_guess', False)
 
    p0,fitfunc,fitfunc_str = common.fit_gauss(fit_offset,fit_amplitude,fit_x0,fit_sigma)

    if show_guess:
        # print decay
        ax.plot(np.linspace(x[0],x[-1],201), fitfunc(np.linspace(x[0],x[-1],201)), ':', lw=2)

    fit_result = fit.fit1d(x,y,None,p0=p0,fitfunc=fitfunc,do_print=True,fixed=fixed,ret=True)

    if isinstance(fit_result, int):
        print "Fit failed!"
       
    return fit_result
   

In [ ]:
start_pos = 2760
end_pos = 2771.0
start_ind = np.argmin(np.abs(b0-start_pos))
end_ind = np.argmin(np.abs(b0-end_pos))

FWHM = 2.83
sigma = FWHM/(2.*np.sqrt(2*np.log(2)))
plt_shift = 2769.55
x0=2769.55-plt_shift
w_start = 2772.85 - plt_shift
w_stop = w_start + 43.5

max_pulse_sig = 0.001

full_hist_data = np.sum(hist_data,axis = 0)

full_hist_data = full_hist_data/float(np.max(full_hist_data))

golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*89
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))

lw = 1
fontsize = 10
markersize = 4

ax = plt.subplot(111)

plt_pos = b0[:-1] + (b0[1]-b0[2])/2.
print plt_pos
plt_pos = plt_pos-plt_shift
print plt_pos
plt.plot(plt_pos,full_hist_data,label='Full signal')

fit_result = fit_gaussian(ax0,plt_pos[start_ind:end_ind],full_hist_data[start_ind:end_ind],fixed = [0,2],fit_x0=x0,fit_sigma = sigma, show_guess = False)

full_gaussian  = fit_result['fitfunc'](plt_pos)       

plt.plot(plt_pos,full_gaussian,label = 'Pulse')
diff_sig = full_hist_data - full_gaussian

for z in range(1,len(plt_pos)):
    relative_signal = np.sum(full_gaussian[-z:])/np.sum(diff_sig[-z:])
    if relative_signal > max_pulse_sig:
        print 'Threshold is at ' ,plt_pos[-z]
        break
plt.plot(plt_pos,diff_sig,label = 'Difference')

###some plot cosmetics
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)

plt.plot([w_start,w_start],[-0.1,1.1],'k--',zorder = 0)
plt.plot([w_stop,w_stop],[-0.1,1.1],'k--',zorder = 0)
ax.legend(borderaxespad=0.5,loc=9,fontsize =fontsize-1,frameon=False)

plt.xlabel('Time (ns)')
plt.ylabel('Norm. counts')
plt.xlim([-10,50])
plt.ylim([-0.1,1.1])
plt.tight_layout()

save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'

plt.savefig(os.path.join(save_folder,'APDTimingHistogram.png'),
                    format='png',bbox_inches = 'tight',pad_inches=0.1)

plt.savefig(os.path.join(save_folder,'APDTimingHistogram.pdf'),
                    format='pdf',bbox_inches = 'tight',pad_inches=0.1)

plt.show()
plt.close('all')

In [ ]:
start_pos = 2760
end_pos = 2771.0
start_ind = np.argmin(np.abs(b0-start_pos))
end_ind = np.argmin(np.abs(b0-end_pos))

FWHM = 2.83
sigma = FWHM/(2.*np.sqrt(2*np.log(2)))

x0=2769.55

max_pulse_sig = 0.001
for i, theta in enumerate(thetas):
    plt_pos = b0[:-1] + (b0[1]-b0[2])/2.
    fig, (ax0) = plt.subplots(figsize=(12,4))
    plt.plot(plt_pos,(hist_data[i]))
    ax0.set_title(theta)
    
    fit_result = fit_gaussian(ax0,plt_pos[start_ind:end_ind],(hist_data[i][start_ind:end_ind]),fixed = [0,2],fit_x0=x0,fit_sigma = sigma, show_guess = False)

    full_gaussian  = fit_result['fitfunc'](plt_pos)       
    
    plt.plot(plt_pos,full_gaussian)
    diff_sig = hist_data[i] - full_gaussian
    
    for z in range(1,len(plt_pos)):
        relative_signal = np.sum(full_gaussian[-z:])/np.sum(diff_sig[-z:])
        if relative_signal > max_pulse_sig:
            print 'Threshold is at ' ,plt_pos[-z]
            break
    plt.plot(plt_pos,diff_sig)
            
    plt.show()
    plt.close('all')
    

In [ ]:
### use these parameters to compare the amount of clicks in the detection window.
start = 2755e3#2762.5e3
window_length = 125e3 # 50e3
hist_binsize = 100
####

reload(pa)
thetas = ['pi/4','pi/5','pi/6','pi/8']
###### get the photon histograms for plu filtered data
offsets_on = True

first_shifted = True
first_unshifted = True
syncs = 0
for i,theta in enumerate(thetas):
    print 'Processing theta ', theta
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,shifted_days = [])
    print len(all_lt4)
    if np.size(all_lt4):
        if not(offsets_on):
            offsets = np.zeros(np.size(all_lt4))
            offsets_ch1 = np.zeros(np.size(all_lt4))
            
        (h0, b0_filt), (h1, b1_filt) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                              start = start,
                                                              length = window_length, 
                                                              hist_binsize = hist_binsize, 
                                                              offsets = offsets, 
                                                              offsets_ch1 = offsets_ch1,
                                                              fltr_plu=True)
        
        syncs += pa.get_no_of_syncs_from_tstamps(all_lt4,folder_lt4)

        if first_shifted:
            ch0_hist_data_unshifted_plu = h0
            ch1_hist_data_unshifted_plu = h1
            first_shifted = False
        else:
            ch0_hist_data_unshifted_plu += h0
            ch1_hist_data_unshifted_plu += h1

            
    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,unshifted_days = [])
    print len(all_lt4)
#     (h0, b0_filt), (h1, b1_filt) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
#                                                           start = start,
#                                                           length = window_length, 
#                                                           hist_binsize = hist_binsize,
#                                                           fltr_plu=True)
    if np.size(all_lt4):
        if not(offsets_on):
            offsets = np.zeros(np.size(all_lt4))
            offsets_ch1 = np.zeros(np.size(all_lt4))
            
        (h0, b0_filt), (h1, b1_filt) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                              start = start,
                                                              length = window_length, 
                                                              hist_binsize = hist_binsize, 
                                                              offsets = offsets, 
                                                              offsets_ch1 = offsets_ch1,
                                                              fltr_plu=True)
        syncs += pa.get_no_of_syncs_from_tstamps(all_lt4,folder_lt4)
        if first_unshifted:
            ch0_hist_data_shifted_plu = h0
            ch1_hist_data_shifted_plu = h1
            first_unshifted = False
        else:
            ch0_hist_data_shifted_plu += h0
            ch1_hist_data_shifted_plu += h1

In [ ]:
print syncs

In [ ]:
fig, (ax0) = plt.subplots(figsize=(12,4))
pa.plot_photon_hist(ax0, ch0_hist_data_unshifted, b0,label = 'ch0',log=True)
pa.plot_photon_hist(ax0, ch1_hist_data_unshifted, b1,label = 'ch1',log=True)
pa.plot_photon_hist(ax0, ch0_hist_data_unshifted_plu, b0_filt,label = 'ch0',log=True)
pa.plot_photon_hist(ax0, ch1_hist_data_unshifted_plu, b1_filt,label = 'ch1',log=True)
ax0.set_title('All data from unshifted days')
plt.show()
plt.close('all')

fig, (ax0) = plt.subplots(figsize=(12,4))
pa.plot_photon_hist(ax0, ch0_hist_data_shifted, b0,label = 'ch0',log=True)
pa.plot_photon_hist(ax0, ch1_hist_data_shifted, b1,label = 'ch1',log=True)
pa.plot_photon_hist(ax0, ch0_hist_data_shifted_plu, b0_filt,label = 'ch0',log=True)
pa.plot_photon_hist(ax0, ch1_hist_data_shifted_plu, b1_filt,label = 'ch1',log=True)
ax0.set_title('All data from shifted days')
plt.show()
plt.close('all')

In [ ]:
reload(pa)
### add all channels together
cum_no_plu = ch0_hist_data_shifted+ch1_hist_data_shifted+ch1_hist_data_unshifted+ch0_hist_data_unshifted
cum_with_plu = ch0_hist_data_shifted_plu+ch1_hist_data_shifted_plu+ch1_hist_data_unshifted_plu+ch0_hist_data_unshifted_plu

golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*120
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))

lw = 1
fontsize = 10
markersize = 4

ax = plt.subplot(111)
pa.plot_photon_hist(ax, cum_no_plu/float(syncs), b0-np.amin(b0),label = 'all events',log=True,normalized= True)
pa.plot_photon_hist(ax, cum_with_plu/float(syncs),b0_filt-np.amin(b0),label = 'triggered events',
                    log=True,normalized=True,lw=1.5)


###some plot cosmetics
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)

ax.legend(borderaxespad=1.,fontsize =fontsize-1,frameon=False)
ax.set_ylim([5e-9,5e-5])
ax.set_xlim([0,120])
plt.xlabel('Time (ns)')
plt.ylabel('Probability per 100 ps bin')

plt.tight_layout()

save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'

plt.savefig(os.path.join(save_folder,'PLU_noPLU_comparison.png'),
                    format='png',bbox_inches = 'tight',pad_inches=0.1)

plt.savefig(os.path.join(save_folder,'PLU_noPLU_comparison.pdf'),
                    format='pdf',bbox_inches = 'tight',pad_inches=0.1)

plt.show()
plt.close('all')

print 'circumvented false detection events'
print sum(cum_no_plu)-sum(cum_with_plu)
print 'Assume that every detection event was in first state generation and include swap time, how much time did we save?'
swap_duration = 34*6.406E-6*2+80E-6
print (sum(cum_no_plu)-sum(cum_with_plu))*swap_duration
print 'cumulative counts no marker ',sum(cum_no_plu)
print 'cumulative counts with marker filter ', sum(cum_with_plu)
print 'fraction of exluded events', 1 - sum(cum_with_plu)/float(sum(cum_no_plu))


# Dark count probability in evaluation window

In [ ]:
### use these parameters to compare the amount of clicks in the detection window.
start = 3000e3#2892.85e3#2762.5e3
window_length = 40e3 # 50e3
hist_binsize = 100
####

reload(pa)
thetas = ['pi/4','pi/5','pi/6','pi/8']
###### get the photon histograms for plu filtered data
offsets_on = True

first_shifted = True
first_unshifted = True

syncs_shifted = 0
syncs_unshifted = 0
for i,theta in enumerate(thetas):
    print 'Processing theta ', theta
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,shifted_days = [])
    print len(all_lt4)
    if np.size(all_lt4):
        if not(offsets_on):
            offsets = np.zeros(np.size(all_lt4))
            offsets_ch1 = np.zeros(np.size(all_lt4))
            
        (h0, b0_dark), (h1, b1_dark) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                              start = start,
                                                              length = window_length, 
                                                              hist_binsize = hist_binsize, 
                                                              offsets = offsets, 
                                                              offsets_ch1 = offsets_ch1,
                                                              fltr_plu=False)

        if first_shifted:
            ch0_hist_data_unshifted_dark = h0
            ch1_hist_data_unshifted_dark = h1
            first_shifted = False
        else:
            ch0_hist_data_unshifted_dark += h0
            ch1_hist_data_unshifted_dark += h1
        
        syncs_shifted += pa.get_no_of_syncs_from_tstamps(all_lt4,folder_lt4)
        
            
    all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True,unshifted_days = [])
    print len(all_lt4)
    (h0, b0_filt), (h1, b1_filt) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                          start = start,
                                                          length = window_length, 
                                                          hist_binsize = hist_binsize,
                                                          fltr_plu=False)
    if np.size(all_lt4):
        if not(offsets_on):
            offsets = np.zeros(np.size(all_lt4))
            offsets_ch1 = np.zeros(np.size(all_lt4))
            
        (h0, b0_dark), (h1, b1_dark) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                              start = start,
                                                              length = window_length, 
                                                              hist_binsize = hist_binsize, 
                                                              offsets = offsets, 
                                                              offsets_ch1 = offsets_ch1,
                                                              fltr_plu=False)
        if first_unshifted:
            ch0_hist_data_shifted_dark = h0
            ch1_hist_data_shifted_dark = h1
            first_unshifted = False
        else:
            ch0_hist_data_shifted_dark += h0
            ch1_hist_data_shifted_dark += h1
            
        syncs_unshifted += pa.get_no_of_syncs_from_tstamps(all_lt4,folder_lt4)
        

In [ ]:
print 'Estimate the dark count probability in a detection window of 40 ns'
cum_ch0_with_dark = ch0_hist_data_shifted_dark+ch0_hist_data_unshifted_dark
cum_ch1_with_dark = ch1_hist_data_shifted_dark+ch1_hist_data_unshifted_dark

print float(np.sum(cum_ch0_with_dark))/(syncs_unshifted+syncs_shifted)
print float(np.sum(cum_ch1_with_dark))/(syncs_unshifted+syncs_shifted)

print 'per second'
dc0 = float(np.sum(cum_ch0_with_dark))/(syncs_unshifted+syncs_shifted)/(1e-12*window_length)
dc1 = float(np.sum(cum_ch1_with_dark))/(syncs_unshifted+syncs_shifted)/(1e-12*window_length)

print dc0, dc1


In [ ]:
### use these parameters to compare the amount of clicks in the detection window.
start = 2755e3#2762.5e3
window_length = 125e3 # 50e3
hist_binsize = 100
####

theta = 'pi/4'
print 'Processing theta ', theta
folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()
Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True)

(h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                      start = start,
                                                      length = window_length, 
                                                      hist_binsize = hist_binsize, 
                                                      offsets = offsets, 
                                                      offsets_ch1 = offsets_ch1,
                                                      fltr_plu=False)

syncs = pa.get_no_of_syncs_from_tstamps(all_lt4,folder_lt4)

cum_plu = h0+h1

pulse_start_ind = np.argmin(abs(b0-2772.95))
interv = b0[1] - b0[0]

darkcounts = (dc0+dc1)*1e-9*interv
print darkcounts
print interv*np.argmin(np.abs(cum_plu[pulse_start_ind:]/float(syncs)-10*darkcounts))


theta = 'pi/8'
print 'Processing theta ', theta
folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()
Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(return_tstamps = True)

(h0, b0), (h1, b1) = pa.get_photon_hists_from_tstamps(all_lt4,folder_lt4,
                                                      start = start,
                                                      length = window_length, 
                                                      hist_binsize = hist_binsize, 
                                                      offsets = offsets, 
                                                      offsets_ch1 = offsets_ch1,
                                                      fltr_plu=False)

syncs = pa.get_no_of_syncs_from_tstamps(all_lt4,folder_lt4)

cum_plu = h0+h1

pulse_start_ind = np.argmin(abs(b0-2772.95))
interv = b0[1] - b0[0]


print interv*np.argmin(np.abs(cum_plu[pulse_start_ind:]/float(syncs)-10*darkcounts))
 